In [1]:
import pandas as pd
import glob
import datetime
import time
import os
from shutil import rmtree

In [2]:
import sys, requests, json, logging, pyotp, time, datetime, pandas
from io import StringIO
import tempfile, os, base64

logging.captureWarnings(True)
##  credential settings, this accont uses OTP, hence the otpsecret is provided

username = ""  ##  You must enter your unique credentials in order to have access to local database
password = ""
secret = ""

def get_new_token():
    auth_tok = None
    if os.path.isfile(TOKEN_PATH):
        token_age = round(time.time() - os.path.getmtime(TOKEN_PATH),0)
        seconds_since_midnight = (now - now.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds()
        if token_age - (86400-seconds_since_midnight) < 0:
            with open(TOKEN_PATH, 'r') as infile:
                # You might consider a test API call to establish token validity here.
                auth_tok = infile.read()
    if not auth_tok:
        auth_server_url = "https://ios.mtnirancell.ir/api/auth/token/2fa/login/"
        verify_url = "https://ios.mtnirancell.ir/api/auth/token/2fa/verify/"
        token_req_payload = {  "password": password,  "username": username}


        token_response = requests.post(auth_server_url,data=token_req_payload, verify=False, allow_redirects=False)
    
    
    
        if token_response.status_code !=200:
            print("Failed to obtain token from the OAuth 2.0 server", file=sys.stderr)
            
        else:
            print("Successfuly authorized")
            totp = pyotp.TOTP(secret) 
            session_id = token_response.json()["session_id"]
            #print(totp.now() )

            request = {  "session_id": session_id,  "code": totp.now()}
            verify_response = requests.post(verify_url,data=request, verify=False, allow_redirects=False)   
            #print(verify_response.json())   
            auth_tok = verify_response.json()['auth_token']
            with open(TOKEN_PATH, 'w') as outfile:
                outfile.write(auth_tok)
    return auth_tok


# SFP Alarm

In [20]:
File = pandas.read_excel('./SFP and Sync.xlsx',sheet_name='SFP ')
AT = pandas.read_excel('./Sites_510.xlsx') #importing list of alarms and other files relating to telecom sites locations
AT = AT[['Site','Name']]
File_M = File.merge(AT,left_on='Site Name',right_on='Site',how='left')
Max = File_M['First Occurred On'].max()
Min = File_M['First Occurred On'].min()
SY = Min.year
SM = Min.month
SD = Min.day
EY = Max.year
EM = Max.month
ED = (Max.day) + 1
Site_List = File_M['Name'].drop_duplicates()
S = Site_List.array
L = S.tolist()

In [21]:

now = datetime.datetime.now()

t_dir = tempfile.TemporaryDirectory(dir = './')
TOKEN_PATH = os.path.join(
    t_dir.name, 
    'ios.token',
)
##
##    function to obtain a new OAuth 2.0 token from the authentication server
##

## 
## 	obtain a token before calling the API for the first time
##
token = get_new_token()
## 
## 	get the list of KPI IDs based on Technology, Vendor and then filter the names
##  the KPI name should be available in selected technology and vendor
##
technology = "LMBB"
vendor = "Nokia"
indicators_url = '' # the URL of the local database
headers  = {'authorization':'Token '+str(token),'Content-Type': 'application/json'}
indicators_request_payload =  {  "technology": technology,  "vendor": vendor}
indicators_response_payload = requests.post(indicators_url,json=indicators_request_payload, verify=False, allow_redirects=False,headers=headers)

df_indicators = pandas.DataFrame(indicators_response_payload.json())

kpis = ['Cell_Availability_All','ERAB_Drop_Rate_Act_All_No_QCI5','Interference_PUSCH_Avg',
        'Erlang_VoLTE','ERAB_Setup_Succ_Rate_Initial','Payload_PDCP_Total_GByte',
        'HOSR_Inter_Freq_Out','HOSR_Intra_Freq_Out','Throughput_UE_QCI9_DL_kbps']

indicators = df_indicators[(df_indicators.name.isin(kpis))].loc[:,"id"].values.tolist()


sdate = datetime.datetime.timestamp(datetime.datetime(SY,SM,SD))*1000  # Importing data of the time period related to alarm list in order to minimize the load on local database
edate = datetime.datetime.timestamp(datetime.datetime(EY,EM,ED))*1000
## 
## 	get the kpi data for selected indicators
##  the KPI name should be available in selected technology and vendor
##

entity_type = "site"
element_list = L # Importing data of every site which is included in the alarm list

kpi_explore_url = ''
headers  = {'authorization':'Token '+token,'Content-Type': 'application/json'}
kpi_request_payload = {"entities":None,
                    "filters":[{"custom":False,"custom_name":None,"element_list":["R5","R10"],"type":"region"}],
                    "indicators":indicators,
                    "layer":None,
                    "technology":technology,
                    "vendor":vendor,
                    "start_date":sdate,
                    "end_date":edate,
                    "granularity":"h",
                    "entity_filters":[{"type":entity_type,"element_list":element_list,"custom":False,"custom_name":None}],
                    "output":"csv"}
kpi_response_payload = requests.post(kpi_explore_url,json=kpi_request_payload, verify=False, allow_redirects=False,headers=headers)

my_string = StringIO(kpi_response_payload.text)
Main_4G = pandas.read_csv(my_string)
# if not df_KPI.empty:
#     print(df_KPI)


Successfuly authorized


In [22]:
Main_4G['time'] = Main_4G['time'].astype('datetime64[ns]')
File_M['First Occurred On'] = File_M['First Occurred On'].dt.round('H')
File_M['Post'] = File_M['First Occurred On'] + pd.Timedelta(hours=2)
File_M['Pre'] = File_M['First Occurred On'] - pd.Timedelta(hours=2)
File_M['Post_U'] = File_M['Post'] + pd.Timedelta(hours=10)
File_M['Pre_U'] = File_M['Pre'] - pd.Timedelta(hours=10)
Set = Main_4G.merge(File_M, left_on=['time','element'], right_on=['First Occurred On','Name'], how='left')
X = Set['element'].unique().tolist()
Y = Set['Alarm Name'].unique().tolist()

In [23]:
POST_F = pandas.DataFrame()
PRE_F = pandas.DataFrame()
CURRENT_F = pandas.DataFrame()
for i in range(len(X)):
    Na = Set[(Set['element'] == X[i]) & (Set['Alarm Name'].isna())]
    for j in range(len(Y)):
        Alarm = Set[(Set['element'] == X[i]) & (Set['Alarm Name'] == Y[j])]
        Alarm = pandas.concat([Alarm,Na])
        Alarm = Alarm.reset_index()
        CUPR = Alarm['Pre'][0]
        Alarm['Pre'] = Alarm['Pre'].fillna(CUPR)
        CUPO = Alarm['Post'][0]
        Alarm['Post'] = Alarm['Post'].fillna(CUPO)
        FOO = Alarm['First Occurred On'][0]
        Alarm['First Occurred On'] = Alarm['First Occurred On'].fillna(FOO)
        ALNM = Alarm['Alarm Name'][0]
        Alarm['Alarm Name'] = Alarm['Alarm Name'].fillna(ALNM)
        ULPR = Alarm['Pre_U'][0]
        Alarm['Pre_U'] = Alarm['Pre_U'].fillna(ULPR)
        ULPO = Alarm['Post_U'][0]
        Alarm['Post_U'] = Alarm['Post_U'].fillna(ULPO)
        PRE = Alarm[(Alarm['time'] >= Alarm['Pre_U']) & (Alarm['time'] < Alarm['Pre'])]
        POST = Alarm[(Alarm['time'] > Alarm['Post']) & (Alarm['time'] <= Alarm['Post_U'])]
        CURRENT = Alarm[(Alarm['time'] >= Alarm['Pre']) & (Alarm['time'] <= Alarm['Post'])]
        
        POST = POST[['element','Alarm Name','First Occurred On','Payload_PDCP_Total_GByte','Throughput_UE_QCI9_DL_kbps','Interference_PUSCH_Avg','ERAB_Setup_Succ_Rate_Initial','Erlang_VoLTE','HOSR_Inter_Freq_Out',
             'HOSR_Intra_Freq_Out','ERAB_Drop_Rate_Act_All_No_QCI5','Cell_Availability_All']].groupby(['element','Alarm Name','First Occurred On']).mean().reset_index()
        PRE = PRE[['element','Alarm Name','First Occurred On','Payload_PDCP_Total_GByte','Throughput_UE_QCI9_DL_kbps','Interference_PUSCH_Avg','ERAB_Setup_Succ_Rate_Initial','Erlang_VoLTE','HOSR_Inter_Freq_Out',
             'HOSR_Intra_Freq_Out','ERAB_Drop_Rate_Act_All_No_QCI5','Cell_Availability_All']].groupby(['element','Alarm Name','First Occurred On']).mean().reset_index()
        CURRENT = CURRENT[['element','Alarm Name','First Occurred On','Payload_PDCP_Total_GByte','Throughput_UE_QCI9_DL_kbps','Interference_PUSCH_Avg','ERAB_Setup_Succ_Rate_Initial','Erlang_VoLTE','HOSR_Inter_Freq_Out',
             'HOSR_Intra_Freq_Out','ERAB_Drop_Rate_Act_All_No_QCI5','Cell_Availability_All']].groupby(['element','Alarm Name','First Occurred On']).mean().reset_index()    
            
        POST_F = pandas.concat([POST_F,POST])
        PRE_F = pandas.concat([PRE_F,PRE])  
        CURRENT_F = pandas.concat([CURRENT_F,CURRENT]) 

In [24]:
Final = PRE_F.merge(POST_F,on=['element','Alarm Name','First Occurred On'],suffixes=('_PRE','_POST'),how='outer').merge(CURRENT_F,on=['element','Alarm Name','First Occurred On'],suffixes=('','_CURRENT'),how='outer')
Final.to_csv('./Input/SFP.csv',index=False)

# Sync Alarm

In [14]:
File = pandas.read_excel('./SFP and Sync.xlsx',sheet_name='Sync ')
File_M = File.merge(AT,left_on='Site Name',right_on='Site',how='left')
Max = File_M['First Occurred On'].max()
Min = File_M['First Occurred On'].min()
SY = Min.year
SM = Min.month
SD = Min.day
EY = Max.year
EM = Max.month
ED = (Max.day) + 1
Site_List = File_M['Name'].drop_duplicates()
S = Site_List.array
L = S.tolist()

In [15]:

now = datetime.datetime.now()

t_dir = tempfile.TemporaryDirectory(dir = './')
TOKEN_PATH = os.path.join(
    t_dir.name, 
    'ios.token',
)
##
##    function to obtain a new OAuth 2.0 token from the authentication server
##

## 
## 	obtain a token before calling the API for the first time
##
token = get_new_token()
## 
## 	get the list of KPI IDs based on Technology, Vendor and then filter the names
##  the KPI name should be available in selected technology and vendor
##
technology = "LMBB"
vendor = "Nokia"
indicators_url = ''
headers  = {'authorization':'Token '+str(token),'Content-Type': 'application/json'}
indicators_request_payload =  {  "technology": technology,  "vendor": vendor}
indicators_response_payload = requests.post(indicators_url,json=indicators_request_payload, verify=False, allow_redirects=False,headers=headers)

df_indicators = pandas.DataFrame(indicators_response_payload.json())

kpis = ['Cell_Availability_All','ERAB_Drop_Rate_Act_All_No_QCI5','Interference_PUSCH_Avg',
        'Erlang_VoLTE','ERAB_Setup_Succ_Rate_Initial','Payload_PDCP_Total_GByte',
        'HOSR_Inter_Freq_Out','HOSR_Intra_Freq_Out','Throughput_UE_QCI9_DL_kbps']

indicators = df_indicators[(df_indicators.name.isin(kpis))].loc[:,"id"].values.tolist()

# duration = 7
# d = datetime.timedelta(days = duration)
# tod = datetime.datetime.now().replace(hour=00, minute=00,second=00,microsecond=00)
# sdate = datetime.datetime.timestamp(tod - d)*1000   #1679689800000
# edate = datetime.datetime.timestamp(tod)*1000       #1679985000000

sdate = datetime.datetime.timestamp(datetime.datetime(SY,SM,SD))*1000
edate = datetime.datetime.timestamp(datetime.datetime(EY,EM,ED))*1000
## 
## 	get the kpi data for selected indicators
##  the KPI name should be available in selected technology and vendor
##

entity_type = "site"
element_list = L

kpi_explore_url = ''
headers  = {'authorization':'Token '+token,'Content-Type': 'application/json'}
kpi_request_payload = {"entities":None,
                    "filters":[{"custom":False,"custom_name":None,"element_list":["R5","R10"],"type":"region"}],
                    "indicators":indicators,
                    "layer":None,
                    "technology":technology,
                    "vendor":vendor,
                    "start_date":sdate,
                    "end_date":edate,
                    "granularity":"h",
                    "entity_filters":[{"type":entity_type,"element_list":element_list,"custom":False,"custom_name":None}],
                    "output":"csv"}
kpi_response_payload = requests.post(kpi_explore_url,json=kpi_request_payload, verify=False, allow_redirects=False,headers=headers)

my_string = StringIO(kpi_response_payload.text)
Main_4G = pandas.read_csv(my_string)
# if not df_KPI.empty:
#     print(df_KPI)


Successfuly authorized


In [16]:
Main_4G['time'] = Main_4G['time'].astype('datetime64[ns]')
File_M['First Occurred On'] = File_M['First Occurred On'].dt.round('H')
File_M['Post'] = File_M['First Occurred On'] + pd.Timedelta(hours=2)
File_M['Pre'] = File_M['First Occurred On'] - pd.Timedelta(hours=2)
File_M['Post_U'] = File_M['Post'] + pd.Timedelta(hours=10)
File_M['Pre_U'] = File_M['Pre'] - pd.Timedelta(hours=10)
Set = Main_4G.merge(File_M, left_on=['time','element'], right_on=['First Occurred On','Name'], how='left')
X = Set['element'].unique().tolist()
Y = Set['Alarm Name'].unique().tolist()

In [17]:
POST_F = pandas.DataFrame()
PRE_F = pandas.DataFrame()
CURRENT_F = pandas.DataFrame()
for i in range(len(X)):
    Na = Set[(Set['element'] == X[i]) & (Set['Alarm Name'].isna())]
    for j in range(len(Y)):
        Alarm = Set[(Set['element'] == X[i]) & (Set['Alarm Name'] == Y[j])]
        Alarm = pandas.concat([Alarm,Na])
        Alarm = Alarm.reset_index()
        CUPR = Alarm['Pre'][0]
        Alarm['Pre'] = Alarm['Pre'].fillna(CUPR)
        CUPO = Alarm['Post'][0]
        Alarm['Post'] = Alarm['Post'].fillna(CUPO)
        FOO = Alarm['First Occurred On'][0]
        Alarm['First Occurred On'] = Alarm['First Occurred On'].fillna(FOO)
        ALNM = Alarm['Alarm Name'][0]
        Alarm['Alarm Name'] = Alarm['Alarm Name'].fillna(ALNM)
        ULPR = Alarm['Pre_U'][0]
        Alarm['Pre_U'] = Alarm['Pre_U'].fillna(ULPR)
        ULPO = Alarm['Post_U'][0]
        Alarm['Post_U'] = Alarm['Post_U'].fillna(ULPO)
        PRE = Alarm[(Alarm['time'] >= Alarm['Pre_U']) & (Alarm['time'] < Alarm['Pre'])]
        POST = Alarm[(Alarm['time'] > Alarm['Post']) & (Alarm['time'] <= Alarm['Post_U'])]
        CURRENT = Alarm[(Alarm['time'] >= Alarm['Pre']) & (Alarm['time'] <= Alarm['Post'])]
        
        POST = POST[['element','Alarm Name','First Occurred On','Payload_PDCP_Total_GByte','Throughput_UE_QCI9_DL_kbps','Interference_PUSCH_Avg','ERAB_Setup_Succ_Rate_Initial','Erlang_VoLTE','HOSR_Inter_Freq_Out',
             'HOSR_Intra_Freq_Out','ERAB_Drop_Rate_Act_All_No_QCI5','Cell_Availability_All']].groupby(['element','Alarm Name','First Occurred On']).mean().reset_index()
        PRE = PRE[['element','Alarm Name','First Occurred On','Payload_PDCP_Total_GByte','Throughput_UE_QCI9_DL_kbps','Interference_PUSCH_Avg','ERAB_Setup_Succ_Rate_Initial','Erlang_VoLTE','HOSR_Inter_Freq_Out',
             'HOSR_Intra_Freq_Out','ERAB_Drop_Rate_Act_All_No_QCI5','Cell_Availability_All']].groupby(['element','Alarm Name','First Occurred On']).mean().reset_index()
        CURRENT = CURRENT[['element','Alarm Name','First Occurred On','Payload_PDCP_Total_GByte','Throughput_UE_QCI9_DL_kbps','Interference_PUSCH_Avg','ERAB_Setup_Succ_Rate_Initial','Erlang_VoLTE','HOSR_Inter_Freq_Out',
             'HOSR_Intra_Freq_Out','ERAB_Drop_Rate_Act_All_No_QCI5','Cell_Availability_All']].groupby(['element','Alarm Name','First Occurred On']).mean().reset_index()    
            
        POST_F = pandas.concat([POST_F,POST])
        PRE_F = pandas.concat([PRE_F,PRE])  
        CURRENT_F = pandas.concat([CURRENT_F,CURRENT]) 

In [18]:
Final = PRE_F.merge(POST_F,on=['element','Alarm Name','First Occurred On'],suffixes=('_PRE','_POST'),how='outer').merge(CURRENT_F,on=['element','Alarm Name','First Occurred On'],suffixes=('','_CURRENT'),how='outer')
Final.to_csv('./Input/Sync.csv',index=False)

In [19]:
path1 = glob.glob('./*tmp*')
for file in path1:
    rmtree(file)